In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

import lightning.pytorch as pl
from tqdm import tqdm as tqdm
import wandb
import lightning.pytorch.loggers as pl_loggers

import sys
sys.path.append('../src/')

%reload_ext autoreload
%autoreload 2

In [2]:
# torch.set_float32_matmul_precision("high")
torch.manual_seed(1337)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn

In [3]:
# Data
from torch.utils.data import random_split

print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)

train_ds, val_ds = random_split(trainset, [45000, 5000])

trainloader = torch.utils.data.DataLoader(
    train_ds, batch_size=128, shuffle=True, num_workers=4)
valloader = torch.utils.data.DataLoader(
    val_ds, batch_size=128, shuffle=False, num_workers=4)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=128, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [4]:
from modules import model_module
from models import resnet

model = model_module.Model(resnet.ResNet18())

In [5]:
wandb_logger = pl_loggers.WandbLogger(name='resnet18', project="td-compression")
# Update some hyperparameters!!!
# wandb_logger.experiment.config.update({key1: val1, key2: val2})

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: usainzg. Use `wandb login --relogin` to force relogin


In [6]:
trainer = pl.Trainer(
    accelerator="gpu", 
    max_epochs=10, 
    logger=wandb_logger
)
trainer.fit(model, train_dataloaders=trainloader, val_dataloaders=valloader)
trainer.test(model, dataloaders=testloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 11.2 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.696    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/usainzg/miniconda3/envs/tdcomp1/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.8417999744415283     │
│         test_loss         │    0.49483808875083923    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.49483808875083923, 'test_accuracy': 0.8417999744415283}]

In [7]:
wandb_logger.experiment.finish()

epoch,▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇█
test_accuracy,▁
test_loss,▁
train_accuracy,▂▁▃▃▄▄▅▄▄▆▅▅▆▆▅▆▇▆▆▇▇█▇▆▇▇█▇▇▇█▇▇▇▇█████
train_loss,▇█▇▆▅▆▄▅▅▄▄▄▃▄▄▃▂▂▃▂▃▁▂▃▂▂▂▂▂▂▁▂▂▂▂▁▁▁▁▂
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
val_accuracy,▁▃▅▆▇▇▇███
val_loss,█▇▄▃▂▃▃▁▁▁
epoch,10
test_accuracy,0.8418
test_loss,0.49484


In [8]:
import copy
import tltorch

factorization = "tucker"
rank = 0.6
decompose_weights = True
td_init = not decompose_weights

decomposition_kwargs = {"init": "random"} if factorization == "cp" else {}
fixed_rank_modes = "spatial" if factorization == "tucker" else None

resnet = resnet.ResNet18()
fact_model = copy.deepcopy(resnet)

layer_names = [
    "layer1.0.conv1",
    "layer1.0.conv2",
    "layer1.1.conv1",
    "layer1.1.conv2",
    "layer2.0.conv1",
    "layer2.0.conv2",
    "layer2.1.conv1",
    "layer2.1.conv2",
    "layer3.0.conv1",
    "layer3.0.conv2",
    "layer3.1.conv1",
    "layer3.1.conv2",
    "layer4.0.conv1",
    "layer4.0.conv2",
    "layer4.1.conv1",
    "layer4.1.conv2",
]

for i, (name, module) in enumerate(resnet.named_modules()):
    if name in layer_names:
        print(f"factorizing: {name}")
        if type(module) == torch.nn.modules.conv.Conv2d:
            fact_layer = tltorch.FactorizedConv.from_conv(
                module,
                rank=rank,
                decompose_weights=decompose_weights,
                factorization=factorization,
                fixed_rank_modes=fixed_rank_modes,
                decomposition_kwargs=decomposition_kwargs,
            )
            if td_init:
                fact_layer.weight.normal_(0, td_init)
            layer, block, conv = name.split(".")
            conv_to_replace = getattr(getattr(fact_model, layer), block)
            setattr(conv_to_replace, conv, fact_layer)

factorizing: layer1.0.conv1
factorizing: layer1.0.conv2
factorizing: layer1.1.conv1
factorizing: layer1.1.conv2
factorizing: layer2.0.conv1
factorizing: layer2.0.conv2
factorizing: layer2.1.conv1
factorizing: layer2.1.conv2
factorizing: layer3.0.conv1
factorizing: layer3.0.conv2
factorizing: layer3.1.conv1
factorizing: layer3.1.conv2
factorizing: layer4.0.conv1
factorizing: layer4.0.conv2
factorizing: layer4.1.conv1
factorizing: layer4.1.conv2


In [16]:
from utils import utils

n_param = utils.count_params(resnet)
n_param_fact = utils.count_params(fact_model)

print(f'original number of parameters: {n_param}')
print(f'factorized number of parameters: {n_param_fact}')
print(f'before - after: {n_param - n_param_fact}')
print(f'compression ratio: {n_param / n_param_fact:.2f}')

original number of parameters: 11173962
factorized number of parameters: 6787383
before - after: 4386579
compression ratio: 1.65


In [9]:
tn_model = model_module.Model(fact_model)

In [10]:
wandb_logger = pl_loggers.WandbLogger(name='tn-resnet18', project="td-compression")
trainer = pl.Trainer(
    accelerator="gpu", 
    max_epochs=10, 
    logger=wandb_logger
)
trainer.fit(tn_model, train_dataloaders=trainloader, val_dataloaders=valloader)
trainer.test(tn_model, dataloaders=testloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 6.8 M 
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
6.8 M     Trainable params
0         Non-trainable params
6.8 M     Total params
27.150    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/usainzg/miniconda3/envs/tdcomp1/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.8281000256538391     │
│         test_loss         │    0.5372909903526306     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.5372909903526306, 'test_accuracy': 0.8281000256538391}]

In [11]:
wandb_logger.experiment.finish()

epoch,▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇█
test_accuracy,▁
test_loss,▁
train_accuracy,▁▂▃▄▅▄▆▆▆▆▆▆▆▅▇▇██▇▇▇▇▇▇▇▇▇▇▇▇██▇▇█▇██▇▇
train_loss,█▇▆▄▄▅▄▄▃▃▃▃▃▄▂▂▂▂▃▂▂▂▂▂▃▂▂▂▂▂▁▁▂▂▂▂▁▂▂▂
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
val_accuracy,▁▅▆▆▇▇█▇██
val_loss,█▃▃▂▂▂▁▂▁▁
epoch,10
test_accuracy,0.8281
test_loss,0.53729


In [12]:
wandb.finish()